## Sample Code

The following is a sample of how to use the different components, as they are kept rather modular in order for you to build pipelines as needed. 

Each class is integrated together into a simple code-generation pipeline using the MAVSdk library. 

In [22]:
# This may be needed to ensure that you have the latest version of the OpenAI library. 
pip install --upgrade openai

SyntaxError: invalid syntax (56689523.py, line 2)

In [24]:
from transcribe import AudioRecorder

# In order to stop the recording => simply stop running the jupyter notebook cell => ctrl + c is used for the terminal specifically. 

audio_recorder = AudioRecorder(duration=5)  # Record for 5 seconds
transcript = audio_recorder.run()
print(transcript)

/Users/alexwang/Humming-Bird/drone/lib/python3.11/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_locati

Listening... Press Ctrl+C to stop.


In [15]:
from plan_agent import DroneCommandPlanner

# This uses the plan_agent, which essentially takes the transcript and converts it into a comprehensive step-by-step plan for the drone to follow.
planner = DroneCommandPlanner()
command = transcript
plan = planner.get_plan(command)
print("Drone Command Plan:\n")
print(plan)


Drone Command Plan:

1. Initialize the communication with the drone using MavSDK library.
2. Take off the drone to hover in the air.
3. Wait for five seconds.
4. Command the drone to follow the specified path or move to a specific location in the air.
5. Provide continuous updates to ensure the drone follows accurately.
6. Monitor the drone's position and adjust its trajectory as needed to maintain the desired follow mode.
7. Ensure the drone stays in control and maintains a safe operational state during the entire process.
8. Upon completion of the task, safely land the drone.


In [16]:
# This is the code agent, which takes the plan and converts it into code that can be run on the drone, this file needs to then by SSH-ed into the drone and run on the drone.
from code_agent import MavSDKCodeGenerator

coder = MavSDKCodeGenerator()
code = coder.generate_code(plan)
print("Generated Code:\n")
print(code)

# This prints a sample of the code. However, the actual code file will be saved as drone_script.py in the current directory.
# Access this, and send that to the drone with SSH. 



Generated Code:

import asyncio
from mavsdk import System
from mavsdk import telemetry

async def run():
    drone = System()
    
    try:
        await drone.connect(system_address="udp://:14540")
        print("Drone connected!")
    
        async for state in drone.core.connection_state():
            if state.is_connected:
                print("Drone discovered!")
                break
        
        async for health in drone.telemetry.health():
            if health.is_gyrometer_calibration_ok:
                print("Drone gyrometer calibrated!")
                break

        await drone.action.arm()
        await drone.action.takeoff()

        
        await asyncio.sleep(5)

        # Specify the path to follow or location to move
        target_location = telemetry.PositionNedYaw(
            5.0, # North position (m)
            5.0, # East position (m)
            -10.0, # Down position (m)
            0.0) # Yaw angle (degrees)
        await drone.action.goto_location